In [3]:
from functools import wraps

# @property
class property(fget=None,fset=None,fdel=None,doc=None)
> 自动创建getter,setter,del，可以对函数属性进行操作

In [3]:
class Student(object):
    @property
    def score(self):
        return self._score

# @wraps
给函数增加额外的功能，但不修改原函数（不破坏所有依赖于原代码的应用程序） 

装饰器会使函数丢失原有的函数名，导致在multiprocessing中不可用，需使用functools.wraps

## 带参数的装饰器
利用\*args,\*\*kargs 

但不识别dataframe这种自定义格式，可以通过封装相关函数避免这个问题

In [4]:
def timer(func):
    @wraps(func)
    def wrapper(*args,**kwargs):
        start_time = time.time()
        func(*args,**kwargs)
        end_time = time.time()
        print("共耗时%s秒" % (end_time - start_time))
    return wrapper

## 带参数的装饰器函数 

In [1]:
def timer(arg):
    def decorator(func):
        @wraps(func)
        def wrapper(*args,**kwargs):
            print("装饰器有个参数%s" %(arg))
            start_time = time.time()
            func(*args,**kwargs)
            end_time = time.time()
            print("共耗时%s秒" % (end_time - start_time))
        return wrapper
    return decorator

## 装饰器函数参数可选 

In [6]:
def timer(arg):
    def decorator(func):
        @wraps(func)
        def wrapper(*args,**kwargs):
            if arg and isinstance(arg, str):
                print("装饰器有个参数%s" %(arg))
            start_time = time.time()
            func(*args,**kwargs)
            end_time = time.time()
            print("共耗时%s秒" % (end_time - start_time))
        return wrapper
    if callable(arg):
        return decorator(arg)
    return decorator

## 注意

In [ ]:
@timer
def countdown(n):
    pass

### 等同于
def countdown(n):
    pass
countdown = timethis(countdown)

# 装饰器定义成类

In [1]:
import types
from functools import wraps

class Profiled:
    def __init__(self, func):
        wraps(func)(self)
        self.ncalls = 0

    def __call__(self, *args, **kwargs):
        self.ncalls += 1
        return self.__wrapped__(*args, **kwargs)

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)

In [2]:
@Profiled
def add(x, y):
    return x + y

In [5]:
add(2, 3)
add.ncalls

3

# 强制函数上的类型检查

In [6]:
from inspect import signature
from functools import wraps

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # If in optimized mode, disable type checking
        if not __debug__:
            return func

        # Map function argument names to supplied types
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments

        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            # Enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(
                            'Argument {} must be {}'.format(name, bound_types[name])
                            )
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [7]:
@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x, y, z)

In [8]:
spam(1, 'hello', 'world')

TypeError: Argument z must be <class 'int'>